In [96]:
##########define states and constants########
gamma = 0.999

#variable 1
grid_length = 100

#variable 2
gas_level_interval = 0.1
num_gas_level_intervals = int(1 / gas_level_interval)

#variable 3
gas_station_present = 2

#variable 4
gas_price_max = 4.00
gas_price_min = 3.00
gas_price_interval = 0.10
num_gas_price_intervals = int((gas_price_max - gas_price_min) / gas_price_interval)

total_num_states = grid_length * grid_length * num_gas_level_intervals 

##########define actions########

#move left, right, up, down
num_moves = 4
#pump or not pump
pump_or_not = 1

total_actions = num_moves + pump_or_not

#########initialization########

#create a 4d matrix, change it to sparse matrix
R = zeros(grid_length,grid_length,num_gas_level_intervals, total_actions)

#randomly pick starting point
start_pt = (int(rand() * grid_length) , int(rand() * grid_length))

#randomly pick ending point
end_pt = (int(rand() * grid_length) , int(rand() * grid_length))
println("end: ", end_pt)

#randomly intialize gas stations
num_gas_stations = 125
station_set = Set()
for i = 1:num_gas_stations
    station_pt = (int(rand() * grid_length) , int(rand() * grid_length))
    push!(station_set,station_pt)
#     while !in( station_pt, station_pt)
#         station_pt = (int(rand() * grid_length) , int(rand() * grid_length))
#         push!(station_set,station_pt)
#     end
end
println("gas: ", station_set)


########initialize rewards#######

#initialize destination reward
destination_default_reward = 6000
dest_offset = 500
for i = 1:num_gas_level_intervals
    gas_level = (i / num_gas_level_intervals)
    R[end_pt[1],end_pt[2],i,:] = destination_default_reward + dest_offset * gas_level
end

#initialize deadend reward
dead_end_reward = -1500
R[:,:, 1,:] = dead_end_reward
#edge case - ensure u can finsih at end point with 0 gas left
gas_level = (1 / num_gas_level_intervals)
R[end_pt[1],end_pt[2],1,:] = destination_default_reward + dest_offset * gas_level

#initialize refuelling reward
refuel_default_reward = 1000
for i = 1:num_gas_level_intervals
#     for j = 1:num_gas_price_intervals
        gas_level = (i / num_gas_level_intervals)
#         curr_price = gas_price_min + (j / num_gas_price_intervals) * (gas_price_max - gas_price_min)
        #action 5 is refuel, 1 means gas station is present
        for pt in station_set
            curr_price = int(rand() * num_gas_price_intervals + 1)
#             R[pt[1],pt[2],i,5] = refuel_default_reward * (1 - gas_level) / curr_price
            R[pt[1],pt[2],i,5] = refuel_default_reward * (1 - gas_level) 
            #if gas is already full, why fill it again?
            if gas_level == 1
                R[pt[1],pt[2],i,5] = -50
            end
        end
#         R[:,:,i,1,j,5] = refuel_default_reward * (1 - gas_level) / curr_price
#         #if gas is already full, why fill it again?
#         if gas_level == 1
#             R[:,:,i,1,j,5] = -100
#         end
#     end
end


end: (75,81)
gas: Set{Any}({(26,40),(10,50),(61,20),(6,48),(15,29),(24,52),(38,60),(49,59),(90,85),(98,40),(76,28),(46,37),(30,94),(16,83),(61,1),(94,63),(15,55),(3,15),(1,33),(72,41),(90,81),(25,68),(94,23),(87,75),(6,90),(23,16),(71,48),(24,68),(94,98),(53,23),(30,61),(43,75),(40,55),(75,60),(26,57),(10,92),(81,61),(58,10),(71,17),(36,33),(60,48),(56,6),(27,82),(17,13),(27,79),(70,23),(34,93),(7,42),(26,100),(59,21),(67,12),(63,6),(63,60),(4,37),(9,18),(58,5),(16,57),(39,58),(57,5),(1,39),(89,84),(12,19),(5,40),(11,13),(39,65),(44,52),(82,87),(77,88),(32,87),(31,37),(36,50),(29,57),(46,19),(73,68),(74,13),(56,70),(90,71),(39,54),(100,67),(97,78),(86,64),(42,96),(35,71),(28,72),(67,77),(60,37),(18,31),(43,12),(44,80),(31,23),(95,67),(35,25),(43,25),(51,10),(49,21),(45,63),(96,66),(98,89),(72,39),(86,86),(78,28),(2,99),(39,49),(1,9),(58,25),(15,6),(20,86),(82,65),(25,73),(66,36),(40,6),(1,29),(84,30),(77,16),(75,72),(44,75),(13,70),(64,28),(60,62),(33,31),(25,25),(57,32),(43,23),(79,58

In [97]:
#########define transtion functions##########

function get_next_states(x, y, glevel, a)
    
    next_state_array = Tuple[]
    
    #move up down left right
    if a == 1 && x + 1 <= grid_length && glevel - 1 >= 1
        push!(next_state_array, (x + 1, y, glevel - 1))
    end
    
    if a == 2 && x - 1 >= 1 && glevel - 1 >= 1
        push!(next_state_array, (x - 1, y, glevel - 1))
    end
    
    if a == 3 && y + 1 <= grid_length && glevel - 1 >= 1
        push!(next_state_array, (x, y + 1, glevel - 1))
    end
    
    if a == 4 && y - 1 >= 1 && glevel - 1 >= 1
        push!(next_state_array, (x, y - 1, glevel - 1))
    end
    
    #topping up gas
    if a == 5 && in((x,y), station_set)
        push!(next_state_array, (x, y, num_gas_level_intervals))
    end
    
#     println((x, y, 1, gas_station_present,num_gas_price_intervals))
    
    return next_state_array
end


get_next_states (generic function with 1 method)

In [98]:
#find all the reachable states from start state
tic()
U = zeros(grid_length,grid_length,num_gas_level_intervals)
A = zeros(grid_length,grid_length,num_gas_level_intervals)
for k = 1:3000
    for i = 1:grid_length
        for j = 1:grid_length
            for glevel = 1:num_gas_level_intervals
                U_max = -99999
                for a = 1:total_actions
                    if a == 5 && !in ((i,j), station_set)
                        continue
                    end
                    next_states_array = get_next_states(i,j,glevel,a)
                    accum_reward = 0
                    for s in next_states_array
                        accum_reward += U[s[1],s[2],s[3]]
                    end
#                     println(accum_reward, "***", next_states_array)
                    curr_u = R[i,j,glevel,a] + gamma * accum_reward
                    if curr_u > U_max
                        U_max = curr_u
                        A[i,j,glevel] = a
                    end
                end
                U[i,j,glevel] = U_max
#                 println(U_max)
            end
        end
#         if i%51 == 0
#             println("halfway")
#         end
    end
    println("k: ",k)
end
toc()

k: 1
k: 2
k: 3
k: 4
k: 5
k: 6
k: 7
k: 8
k: 9
k: 10
k: 11
k: 12
k: 13
k: 14
k: 15
k: 16
k: 17
k: 18
k: 19
k: 20
k: 21
k: 22
k: 23
k: 24
k: 25
k: 26
k: 27
k: 28
k: 29
k: 30
k: 31
k: 32
k: 33
k: 34
k: 35
k: 36
k: 37
k: 38
k: 39
k: 40
k: 41
k: 42
k: 43
k: 44
k: 45
k: 46
k: 47
k: 48
k: 49
k: 50
k: 51
k: 52
k: 53
k: 54
k: 55
k: 56
k: 57
k: 58
k: 59
k: 60
k: 61
k: 62
k: 63
k: 64
k: 65
k: 66
k: 67
k: 68
k: 69
k: 70
k: 71
k: 72
k: 73
k: 74
k: 75
k: 76
k: 77
k: 78
k: 79
k: 80
k: 81
k: 82
k: 83
k: 84
k: 85
k: 86
k: 87
k: 88
k: 89
k: 90
k: 91
k: 92
k: 93
k: 94
k: 95
k: 96
k: 97
k: 98
k: 99
k: 100
k: 101
k: 102
k: 103
k: 104
k: 105
k: 106
k: 107
k: 108
k: 109
k: 110
k: 111
k: 112
k: 113
k: 114
k: 115
k: 116
k: 117
k: 118
k: 119
k: 120
k: 121
k: 122
k: 123
k: 124
k: 125
k: 126
k: 127
k: 128
k: 129
k: 130
k: 131
k: 132
k: 133
k: 134
k: 135
k: 136
k: 137
k: 138
k: 139
k: 140
k: 141
k: 142
k: 143
k: 144
k: 145
k: 146
k: 147
k: 148
k: 149
k: 150
k: 151
k: 152
k: 153
k: 154
k: 155
k: 156
k: 157
k: 158
k:

892.380393757

In [ ]:
(grid_length,grid_length,num_gas_level_intervals,gas_station_present,num_gas_price_intervals, total_actions), total_num_states

In [ ]:
#begin value iteration

In [ ]:
nextd_states = [1 2 3]

In [ ]:
R

In [ ]:
R[end_pt[1],end_pt[2],:,:,:]

In [77]:
U[end_pt[1]-1,end_pt[2],:,:]

1x1x10x1 Array{Float64,4}:
[:, :, 1, 1] =
 -1000.0

[:, :, 2, 1] =
 2897.5

[:, :, 3, 1] =
 2042.5

[:, :, 4, 1] =
 5607.49

[:, :, 5, 1] =
 4883.36

[:, :, 6, 1] =
 9131.81

[:, :, 7, 1] =
 9148.36

[:, :, 8, 1] =
 11424.0

[:, :, 9, 1] =
 11486.4

[:, :, 10, 1] =
 13587.6

In [99]:
println(end_pt[1]," ",end_pt[2])
f = open("/Users/yishh/Box Sync/CS 238/project_stuff/proj_utility_output_1.csv","w")
a_f = open("/Users/yishh/Box Sync/CS 238/project_stuff/proj_actions_output_1.csv","w")
for i = 1:grid_length
    for j = 1:grid_length
        #get the max of each x,y coordinate
        #find the action
#         print(maximum(U[i,j,:,:,:]), " ")
#         write(f, string(round(maximum(U[i,j,:]),1)) ,"-", string(indmax(U[i,j,:])))
         write(f, string(round(maximum(U[i,j,:]),1)) )
        write(f, "\t")
        write(a_f, string(int(A[i,j,:][indmax(U[i,j,:])])))
        write(a_f, "\t")
    end
#     println("")
    write(f, "\n")
    write(a_f, "\n")
end
close(f)
close(a_f)

75 81


In [ ]:
maximum(U[end_pt[1],end_pt[2],:,:])

In [ ]:
R[end_pt[1]-1,end_pt[2],:,1]

In [ ]:
U[end_pt[1]-1,end_pt[2],:]

In [ ]:
indmax(U[11,7,:])

In [ ]:
A[10,10,:]

In [ ]:
end_pt[1]-2,end_pt[2]

In [ ]:
station_set

In [48]:
end_pt[1],end_pt[2]

(2,52)

In [49]:
U[end_pt[1]-1,end_pt[2],:]

1x1x10 Array{Float64,3}:
[:, :, 1] =
 -1000.0

[:, :, 2] =
 155.0

[:, :, 3] =
 0.0

[:, :, 4] =
 320.0

[:, :, 5] =
 170.0

[:, :, 6] =
 495.0

[:, :, 7] =
 350.0

[:, :, 8] =
 680.0

[:, :, 9] =
 1292.0

[:, :, 10] =
 1295.0

In [ ]:
A[end_pt[1]-2,end_pt[2],:][indmax(U[end_pt[1]-2,end_pt[2],:])]

In [ ]:

A[2,2,:][indmax(U[2,2,:])]

In [ ]:
A[2,2,:]

In [ ]:
R[2,2,:]

In [ ]:
end_pt

In [60]:
station_set

Set{Any}({(22,3),(92,38),(30,3),(12,57),(95,63),(30,69),(87,56),(97,87),(98,1),(21,41),(47,98),(82,48),(72,100),(58,81),(42,66),(49,90),(92,77),(70,30),(36,30),(2,58),(26,75),(3,18),(86,14),(59,61),(89,43),(85,74),(70,60),(73,58),(10,19),(59,67),(42,55),(95,97),(48,2),(31,34),(19,23),(40,39),(55,35),(37,59),(70,88),(40,37),(88,60),(31,96),(78,100),(93,48),(80,55),(81,65),(96,24),(60,26),(97,70),(35,2),(52,55),(4,91),(22,54),(75,97),(43,8),(78,61),(7,30),(16,69),(17,8),(42,23),(21,95),(40,38),(31,97),(59,66),(35,27),(68,63),(58,47),(2,57),(69,85),(74,55),(100,73),(41,62),(27,53),(12,67),(1,85),(21,97),(98,63),(92,30),(19,78),(5,47),(97,80),(50,32),(35,57),(28,81),(16,24),(69,18),(68,66),(55,24),(22,94),(91,83),(43,12),(93,86),(71,80),(45,46),(70,9),(45,38),(54,40),(100,76),(10,73),(69,48),(18,99),(88,49),(31,42),(57,67),(17,27),(31,15),(64,23),(20,36),(11,82),(25,73),(94,49),(45,13),(82,18),(83,81),(62,34),(88,96),(64,69),(17,38),(20,94),(85,51),(50,96),(73,92),(57,80),(56,82)})

In [100]:
end_pt

(75,81)

In [104]:
function bfs(x,y)
    first_action = A[x,y,10]
    next_state = get_next_state(x,y,10,first_action)
    println(next_state)
    for i = 1:10000
        next_state = get_next_state(next_state[1],next_state[2],next_state[3],A[next_state[1],next_state[2],next_state[3]])
        
        println(next_state, "   U: ", U[next_state[1], next_state[2],next_state[3]], "    a: ", A[next_state[1],next_state[2],next_state[3]])
        if (next_state[1],next_state[2]) == end_pt
            println("reached goal")
            break
        end
    end
end

bfs (generic function with 2 methods)

In [106]:
# A[34,4,10]
bfs(74,74)

(75,74,9)
(76,74,8)   U: 88343.4139996316    a: 1.0
(77,74,7)   U: 88432.70264197077    a: 1.0
(78,74,6)   U: 88522.0789059345    a: 2.0
(77,74,5)   U: 88610.68959553003    a: 2.0
(76,74,4)   U: 88699.38898451455    a: 2.0
(75,74,3)   U: 88788.17716167623    a: 4.0
(75,73,2)   U: 88877.05421589212    a: 4.0
(75,72,1)   U: 88966.02023612824    a: 5.0
(75,72,10)   U: 88155.03198246052    a: 1.0
(76,72,9)   U: 88243.27525771824    a: 1.0
(77,72,8)   U: 88332.4443474756    a: 1.0
(78,72,7)   U: 88421.72286683612    a: 1.0
(79,72,6)   U: 88511.08899592994    a: 2.0
(78,72,5)   U: 88599.68868461455    a: 2.0
(77,72,4)   U: 88688.37706167622    a: 2.0
(76,72,3)   U: 88777.15421589211    a: 2.0
(75,72,2)   U: 88866.02023612824    a: 5.0
(75,72,10)   U: 88155.03198246052    a: 1.0
(76,72,9)   U: 88243.27525771824    a: 1.0
(77,72,8)   U: 88332.4443474756    a: 1.0
(78,72,7)   U: 88421.72286683612    a: 1.0
(79,72,6)   U: 88511.08899592994    a: 2.0
(78,72,5)   U: 88599.68868461455    a: 2.0
(77

In [10]:
#########define transtion functions##########

function get_next_state(x, y, glevel, a)
    
    next_state_array = Tuple[]
    
    #move up down left right
    if a == 1 && x + 1 <= grid_length && glevel - 1 >= 1
        return (x + 1, y, glevel - 1)
    end
    
    if a == 2 && x - 1 >= 1 && glevel - 1 >= 1
        return x - 1, y, glevel - 1
    end
    
    if a == 3 && y + 1 <= grid_length && glevel - 1 >= 1
        return x, y + 1, glevel - 1
    end
    
    if a == 4 && y - 1 >= 1 && glevel - 1 >= 1
        return x, y - 1, glevel - 1
    end
    
    #topping up gas
    if a == 5 && in((x,y), station_set)
        return x, y, num_gas_level_intervals
    end
    
#     println((x, y, 1, gas_station_present,num_gas_price_intervals))
    return 0
    return next_state_array
end


get_next_state (generic function with 1 method)

In [84]:
end_pt

(74,76)